In [1]:
from pymongo import MongoClient
import pandas as pd

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold


############

In [ ]:
conn = MongoClient('140.115.53.147', 27017)
conn.database_names()
datasets ='NCUx107_Pre_college_calculas'
db = conn[datasets]


In [ ]:
collection = db['students_grades']
score_df = pd.DataFrame(list(collection.find()))
score_df=score_df[['studentId','finalScore']]
len(score_df)

In [ ]:
collection = db['ma_w8_avg.csv']
feature_df = pd.DataFrame(list(collection.find()))
feature_df = feature_df.drop(['_id','email','course_id'], axis=1)
feature_df = feature_df.fillna('0')
feature_df = feature_df.rename(columns={'username':'studentId'})
feature_df.head(5)


In [2]:
NCUx107_Pre=pd.merge(score_df,feature_df,on='studentId')
NCUx107_Pre=NCUx107_Pre.drop(['studentId'], axis=1)


NCUx107_Pre['class'] = NCUx107_Pre['finalScore'] < 60
NCUx107_Pre = NCUx107_Pre.drop(['finalScore'], axis=1)
NCUx107_Pre


NameError: name 'score_df' is not defined

In [3]:
X = NCUx107_Pre.drop(['class'], axis=1)
y = NCUx107_Pre['class']
X, y = X[y != 2], y[y != 2]
n_samples, n_features = X.shape

# Add noisy features
random_state = np.random.RandomState(0)
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# #############################################################################
# Classification and ROC analysis

# Run classifier with cross-validation and plot ROC curves

cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)


NameError: name 'NCUx107_Pre' is not defined

In [ ]:


tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

plt.figure(figsize=(10, 5))

i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()








